In [19]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from datetime import datetime
from openpyxl import load_workbook
from utils import append_df_to_excel, threshold_rmse
from xgboost import XGBRegressor
from autofeat import AutoFeatRegressor

In [20]:
# gleiche fehlerhafte no2 oder sonstige werte kicken
# nur eines der neuen features verweden
# outliers entfernen
# featuretools richtig mit neuen columns mean etc

## 2. Parameters & Reading in the data

### 2.1 Reading in the data

In [21]:
# read in prepared data
df_to_predict_16 = pd.read_excel("prepared_data22.xlsx", sheet_name = "Sensor_ID_16")
df_to_predict_20 = pd.read_excel("prepared_data22.xlsx", sheet_name = "Sensor_ID_20")
df_test_16 = pd.read_excel("prepared_data22.xlsx", sheet_name = "Sensor_ID_16_test")
df_test_20 = pd.read_excel("prepared_data22.xlsx", sheet_name = "Sensor_ID_20_test")

# Randomize Order of rows for better equality in Cross validation splitting
df_to_predict_16 = df_to_predict_16.sample(random_state = 60, frac = 1)
df_to_predict_20 = df_to_predict_20.sample(random_state = 60, frac = 1)

# create X & Y Data for Sensor = 16
Y_16 = df_to_predict_16["no2"]
X_16 = df_to_predict_16.drop(columns = ["no2"])

# create X & Y Data for Sensor = 20
Y_20 = df_to_predict_20["no2"]
X_20 = df_to_predict_20.drop(columns = ["no2"])


#print(Y_16, Y_20)
#X_16.head()
X_16

,t_degree_c,h_percent,p_mbar,tavg,tmin,tmax,prcp,snow,wdir,wspd,...,year,month,day,hour,weekday,weeknumber,is_workday,quarter,12_o_clock_no2,t_degree_c_squared
4153,23.116667,47.250000,1004.191667,19.4,14.7,24.4,0.0,0.0,132.0,10.1,...,2020,7,17,16,4,29,1,3,14.342250,534.380278
2035,10.533333,28.000000,1010.400000,5.8,-0.6,9.7,0.0,0.0,75.0,17.6,...,2020,3,31,14,1,14,1,1,0.212478,110.951111
5338,19.958333,76.775000,1002.800000,23.6,15.1,32.7,0.0,0.0,118.0,8.6,...,2020,9,16,1,2,38,1,3,62.627825,398.335069
7108,3.791667,99.900000,1008.991667,4.3,2.2,5.3,0.0,0.0,152.0,5.8,...,2020,12,18,9,4,51,1,4,22.947600,14.376736
1787,11.783333,81.275000,1014.250000,11.1,5.2,17.2,0.0,0.0,242.0,7.2,...,2020,3,17,21,1,12,1,1,18.644925,138.846944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4168,20.550000,64.691667,1004.616667,21.3,15.1,27.5,0.0,0.0,141.0,7.2,...,2020,7,18,7,5,29,0,3,57.528358,422.302500
6543,5.858333,93.408333,1018.550000,3.3,-1.8,6.9,0.0,0.0,245.0,8.6,...,2020,11,20,16,4,47,1,4,34.262042,34.320069
1418,2.100000,95.480000,1000.040000,4.6,0.8,7.6,1.7,10.0,211.0,17.6,...,2020,2,28,3,4,9,1,1,24.668670,4.410000
3137,12.391667,89.500000,1009.750000,13.9,8.7,17.6,1.1,0.0,258.0,15.1,...,2020,5,23,13,5,21,0,2,24.222467,153.553403


In [4]:
#X_20.drop(columns=["12_o_clock_no2"], inplace = True)

In [5]:
#X_20.info()

## Automated Feature Engineering

In [6]:
#model = AutoFeatRegressor()

In [7]:
#model.fit_transform(X_20, Y_20)

### 2.2 Parameters about the prepared data

In [8]:
# Set Parameters to save information about the prepared data that the models are working with
DESCRIPTION = "Test with 12 uhr no2 werte feature"
NAN_VALUES_FILLED_WITH = "Monthly average"
DATA_NORMALIZED = False
DELETED_VALUES = ""
FEATURES_USED = ", ".join([feature for feature in df_to_predict_16.columns])

DATE = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#  pipeline should be able to test multiple models
#  evaluate them with RMSE
#  Save the result with date in an excel sheet

In [33]:
# Initialize empty Model Result List
model_results = list()

## 3. XGBoost Regression

#### 3.1 Sensor_ID 16

In [22]:
# Sensor_ID = 16

XGB_pipe_16 = Pipeline([
    ("model", XGBRegressor())
])
XGB_pipe_16.get_params()

XGB_model_16 = GridSearchCV(estimator = XGB_pipe_16,
                                    n_jobs = -1,
                                    param_grid = {'model__n_estimators': list([2000,3000]), 'model__max_depth': list(range(9,11))},
                                    scoring = {"threshold_rmse": make_scorer(threshold_rmse, greater_is_better=True),
                                              "r2_score": make_scorer(r2_score),
                                              "MAE": make_scorer(mean_absolute_error)},
                                    cv = 10,
                                    refit = "threshold_rmse")

XGB_model_16.fit(X_16, Y_16)




GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('model',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     gamma=None, gpu_id=None,
                                                     importance_type='gain',
                                                     interaction_constraints=None,
                                                     learning_rate=None,
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     min_child_weight=None,
                                   

#### 3.2 Sensor_ID 20

In [23]:
# Sensor_ID = 20

XGB_pipe_20 = Pipeline([
    ("model", XGBRegressor())
])
#Linear_Regession_pipe.get_params()

XGB_model_20 = GridSearchCV(estimator = XGB_pipe_20,
                                    n_jobs = -1,
                                    param_grid = {'model__n_estimators': list([2000,3000]), 'model__max_depth': list(range(5,7))},
                                    scoring = {"threshold_rmse": make_scorer(threshold_rmse, greater_is_better=True),
                                              "r2_score": make_scorer(r2_score),
                                              "MAE": make_scorer(mean_absolute_error)},
                                    cv = 10,
                                    refit = "threshold_rmse")

XGB_model_20.fit(X_20, Y_20)



GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('model',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     gamma=None, gpu_id=None,
                                                     importance_type='gain',
                                                     interaction_constraints=None,
                                                     learning_rate=None,
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     min_child_weight=None,
                                   

In [15]:
XGB_model_16.best_estimator_

Pipeline(steps=[('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=9, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=2000,
                              n_jobs=12, num_parallel_tree=1, random_state=0,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exact',
                              validate_parameters=1, verbosity=None))])

In [16]:
XGB_model_20.best_estimator_

Pipeline(steps=[('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=3000,
                              n_jobs=12, num_parallel_tree=1, random_state=0,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exact',
                              validate_parameters=1, verbosity=None))])

## 4. Model evaluation

In [30]:
XGB_results_16.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_model__max_depth', 'param_model__n_estimators', 'params',
       'split0_test_threshold_rmse', 'split1_test_threshold_rmse',
       'split2_test_threshold_rmse', 'split3_test_threshold_rmse',
       'split4_test_threshold_rmse', 'split5_test_threshold_rmse',
       'split6_test_threshold_rmse', 'split7_test_threshold_rmse',
       'split8_test_threshold_rmse', 'split9_test_threshold_rmse',
       'mean_test_threshold_rmse', 'std_test_threshold_rmse',
       'rank_test_threshold_rmse', 'split0_test_r2_score',
       'split1_test_r2_score', 'split2_test_r2_score', 'split3_test_r2_score',
       'split4_test_r2_score', 'split5_test_r2_score', 'split6_test_r2_score',
       'split7_test_r2_score', 'split8_test_r2_score', 'split9_test_r2_score',
       'mean_test_r2_score', 'std_test_r2_score', 'rank_test_r2_score',
       'split0_test_MAE', 'split1_test_MAE', 'split2_test_MAE',
       'split3_test_MA

In [89]:
#XGB_results_16[["params","mean_test_threshold_rmse"]]

In [35]:
XGB_results_16 = pd.DataFrame(XGB_model_16.cv_results_)
XGB_results_20 = pd.DataFrame(XGB_model_20.cv_results_)

# Append results to result list
model_results.append(XGB_results_16["mean_test_r2_score"].max())
model_results.append(XGB_results_16["mean_test_threshold_rmse"].min())
model_results.append(XGB_results_16["mean_test_MAE"].min())
model_results.append(XGB_results_20["mean_test_r2_score"].max())
model_results.append(XGB_results_20["mean_test_threshold_rmse"].min())
model_results.append(XGB_results_20["mean_test_MAE"].min())

In [25]:
feature_scores_20 = XGB_model_20.best_estimator_.named_steps["model"].feature_importances_
feature_scores_16 = XGB_model_20.best_estimator_.named_steps["model"].feature_importances_
#print(XGB_model_20.best_estimator_.named_steps["model"].booster())
#XGB_results_16[["mean_test_threshold_rmse"]]
for column, feature_importance in zip(X_20.columns, feature_scores_20):
    print(column, feature_importance)

t_degree_c 0.12882149
h_percent 0.0076997243
p_mbar 0.005203964
tavg 0.008451054
tmin 0.022126883
tmax 0.022561338
prcp 0.016286505
snow 0.01704287
wdir 0.022342483
wspd 0.02121407
wpgt 0.015103899
pres 0.014770839
tsun 0.02890913
A81_D44_Z 0.09427595
year 0.010088597
month 0.097313054
day 0.015836379
hour 0.051899005
weekday 0.021488998
weeknumber 0.02881068
is_workday 0.0
quarter 0.0
12_o_clock_no2 0.33924797
t_degree_c_squared 0.010505074


## 5. Writing Results to Excel

In [18]:
arrays = [list(4*"A"),
          list(4*"B"), 
          list(4*"C")]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=["Algorithm", "Sensor_ID", "Metric"])
df_results = pd.DataFrame(np.array([model_results]), columns=index[:20])

df_results.insert(0, 'Feaatures used', FEATURES_USED)
df_results.insert(0, 'Data Normalized', DATA_NORMALIZED)
df_results.insert(0, 'Deleted Values', DELETED_VALUES)
df_results.insert(0, 'NaN filled with', NAN_VALUES_FILLED_WITH)
df_results.insert(0, 'Description', DESCRIPTION)
df_results.insert(0, 'Date', DATE)

df_results

Algorithm                 Date                         Description  \
Sensor_ID                                                            
Metric                                                               
0          2021-12-07 17:00:46  Test with 12 uhr no2 werte feature   

Algorithm  NaN filled with Deleted Values Data Normalized  \
Sensor_ID                                                   
Metric                                                      
0          Monthly average                          False   

Algorithm                                     Feaatures used         A  \
Sensor_ID                                                            B   
Metric                                                               C   
0          t_degree_c, h_percent, p_mbar, tavg, tmin, tma...  0.886803   

Algorithm                                 
Sensor_ID                                 
Metric            C         C          C  
0          8.505784  0.931217  17.439369

In [36]:
model_results

[0.8868034649212824,
 8.505784164920552,
 4.185902106982027,
 0.9312165047741727,
 17.439368929504177,
 6.969866371013967]

In [93]:
filename = r'Model_Results_XGBoost_12.xlsx'

#append_df_to_excel(filename, df_model_results)

append_df_to_excel(filename, df_results, sheet_name='Model_Results')

#append_df_to_excel(filename, df_model_results, sheet_name='Sheet2', index=False)

#append_df_to_excel(filename, df_model_results, sheet_name='Sheet2', index=False, startrow=25)

ValueError: Sheet 'Model_Results' already exists and if_sheet_exists is set to 'error'.

In [ ]:
df_test

In [68]:
df_test_20

,t_degree_c,h_percent,p_mbar,tavg,tmin,tmax,prcp,snow,wdir,wspd,...,month,day,hour,weekday,weeknumber,is_workday,quarter,12_o_clock_no2,t_degree_c_squared,no2
index,,,,,,,,,,,,,,,,,,,,,
-100000,3.216667,95.000000,996.566667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12,11,17,2,50,1,4,67.089858,10.346944,69.115585
-99999,2.508333,96.666667,996.416667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12,11,18,2,50,1,4,67.089858,6.291736,56.625820
-99997,5.300000,84.008333,988.266667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12,12,17,3,50,1,4,44.142258,28.090000,58.243797
-99996,5.108333,83.041667,987.050000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12,12,18,3,50,1,4,44.142258,26.095069,52.105068
-99993,8.900000,71.216667,989.191667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12,14,17,5,50,0,4,47.010708,79.210000,65.715042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-99295,8.541667,60.200000,1021.733333,5.9,-0.1,10.6,0.0,0.0,42.0,10.1,...,2,28,18,6,8,0,1,29.800008,72.960069,46.717896
-99294,7.000000,37.216667,1017.683333,1.8,-3.9,7.9,0.0,0.0,36.0,7.9,...,3,6,17,5,9,0,1,22.469525,49.000000,47.302113
-99292,5.408333,44.100000,1017.716667,1.8,-3.9,7.9,0.0,0.0,36.0,7.9,...,3,6,18,5,9,0,1,22.469525,29.250069,49.340122


In [63]:
df_test_20 = df_test_20.set_index("index", drop = True)
df_test_16 = df_test_16.set_index("index", drop = True)

In [64]:
df_test_16.index = df_test_16.index - 100000

In [65]:
df_test_20.index = df_test_20.index - 100000

In [66]:
df_test_20

,t_degree_c,h_percent,p_mbar,tavg,tmin,tmax,prcp,snow,wdir,wspd,...,year,month,day,hour,weekday,weeknumber,is_workday,quarter,12_o_clock_no2,t_degree_c_squared
index,,,,,,,,,,,,,,,,,,,,,
0,3.216667,95.000000,996.566667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019,12,11,17,2,50,1,4,67.089858,10.346944
1,2.508333,96.666667,996.416667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019,12,11,18,2,50,1,4,67.089858,6.291736
3,5.300000,84.008333,988.266667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019,12,12,17,3,50,1,4,44.142258,28.090000
4,5.108333,83.041667,987.050000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019,12,12,18,3,50,1,4,44.142258,26.095069
7,8.900000,71.216667,989.191667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019,12,14,17,5,50,0,4,47.010708,79.210000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,8.541667,60.200000,1021.733333,5.9,-0.1,10.6,0.0,0.0,42.0,10.1,...,2021,2,28,18,6,8,0,1,29.800008,72.960069
706,7.000000,37.216667,1017.683333,1.8,-3.9,7.9,0.0,0.0,36.0,7.9,...,2021,3,6,17,5,9,0,1,22.469525,49.000000
708,5.408333,44.100000,1017.716667,1.8,-3.9,7.9,0.0,0.0,36.0,7.9,...,2021,3,6,18,5,9,0,1,22.469525,29.250069


In [67]:
predicted_20 = XGB_model_20.predict(df_test_20)
predicted_16 = XGB_model_16.predict(df_test_16)

In [68]:
len(predicted_16)

356

In [69]:
df_test_20["no2"] = predicted_20
df_test_16["no2"] = predicted_16

In [70]:
df_test_final_20 = df_test_20.filter(["no2"])

In [71]:
df_test_final_16 = df_test_16.filter(["no2"])

In [72]:
df_test_final = pd.concat([df_test_final_20, df_test_final_16])

In [73]:
df_test_final.index.rename('id', inplace = True)

In [74]:
df_test_final = df_test_final.sort_index()

In [75]:
df_test_final

,no2
id,
0,53.715946
1,48.184330
2,11.928917
3,50.678226
4,48.980694
...,...
709,39.264046
710,50.060478
711,49.682617


In [76]:
df_test_final.to_csv("submission20.csv")

In [107]:
df = pd.read_csv("best_submission.csv")
df

,id,no2
0,0,68.444084
1,1,62.938770
2,2,10.789433
3,3,49.201893
4,4,47.719444
...,...,...
709,709,37.482520
710,710,64.185520
711,711,48.728188
712,712,64.403680


In [108]:
df[df["no2"] < 30 ] += 3

In [109]:
df[df["no2"] > 60] -= 4.5

In [110]:
df["id"] = df.index

In [111]:
df

,id,no2
0,0,63.944084
1,1,58.438770
2,2,13.789433
3,3,49.201893
4,4,47.719444
...,...,...
709,709,37.482520
710,710,59.685520
711,711,48.728188
712,712,59.903680


In [183]:
df["no2"] = df["no2"] + 4.5

In [112]:
df.set_index("id", drop=True, inplace = True)

In [113]:
df

,no2
id,
0,63.944084
1,58.438770
2,13.789433
3,49.201893
4,47.719444
...,...
709,37.482520
710,59.685520
711,48.728188


In [115]:
df.to_csv("submission22.csv")

In [185]:
df
#currently all values -4.5
# values über 60 --> erhöhren 
# values unter 60 eher verringern (-3?)

,no2
id,
0,72.944084
1,67.438770
2,15.289433
3,53.701893
4,52.219444
...,...
709,41.982520
710,68.685520
711,53.228188
